## 認証とAPIのビルドと初期化

In [1]:
import os
import pickle

from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

In [2]:
def get_credentials(client_secret_file, scopes,
                    token_storage_pkl='token.pickle'):
    '''google_auth_oauthlibを利用してOAuth2認証

        下記URLのコードをほぼそのまま利用。Apache 2.0
        https://developers.google.com/drive/api/v3/quickstart/python#step_1_turn_on_the_api_name
    '''
    creds = None
    # token.pickleファイルにユーザのアクセス情報とトークンが保存される
    # ファイルは初回の認証フローで自動的に作成される
    if os.path.exists(token_storage_pkl):
        with open(token_storage_pkl, 'rb') as token:
            creds = pickle.load(token)
            
    # 有効なクレデンシャルがなければ、ユーザーにログインしてもらう
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                client_secret_file, scopes=scopes)
            creds = flow.run_local_server(port=0)
            
        # クレデンシャルを保存（次回以降の認証のため）
        with open(token_storage_pkl, 'wb') as token:
            pickle.dump(creds, token)

    return creds

In [3]:
'''OAuth認証とAPIのビルド実行'''

# 利用するAPIサービス
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

# OAuthのスコープとクレデンシャルファイル
YOUTUBE_READ_WRITE_SCOPE = 'https://www.googleapis.com/auth/youtube'
CLIENT_SECRET_FILE = 'config/client_secret.json'

# OAuth認証：クレデンシャルを作成
creds = get_credentials(
                    client_secret_file=CLIENT_SECRET_FILE,
                    scopes=YOUTUBE_READ_WRITE_SCOPE
                    )

# API のビルドと初期化
youtube_auth = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
                    credentials=creds)


## プレイリスト作成

In [4]:
'''新規再生リストの作成'''

title = 'B-life Test 10分〜15分'
description = '再生時間が10分〜15分の動画のプレイリスト'
privacy_status = 'public'

# 新規プレイリストを追加
# https://developers.google.com/youtube/v3/docs/playlists/insert
playlists_insert_response = youtube_auth.playlists().insert(
  part="snippet, status",
  body=dict(
    snippet=dict(
      title=title,
      description=description
    ),
    status=dict(
      privacyStatus=privacy_status
    )
  )
).execute()


In [5]:
'''再生リストへの動画の追加 '''

# 動画ID
# videoids = df_video_playlist['id'].values

videoids = ['nZSe3ZZUSJw', 'EYqPH91q5nE', 'Zw_osQctQKs', 'kBLCmz8pODg',
                     'IBt5l9Um_rY', 'WWxKW8ncPe0', 'CmWJJWKLeKg', 'e9HR7-3I3MA',]

# プレイリストに動画を追加
# https://stackoverflow.com/questions/20650415/insert-video-into-a-playlist-with-youtube-api-v3/22190766

playlistid = playlists_insert_response['id']  # 作成した再生リストのIDを取得

# 動画IDをループ
for videoid in videoids:
  
    resourceid = dict(kind='youtube#video',
                      videoId=videoid)

    response = youtube_auth.playlistItems().insert(
                part='snippet',
                  body=dict(
                    snippet=dict(
                      playlistId=playlistid,
                      resourceId=resourceid
                      )
                   )
            ).execute()